In [ ]:
### NOTEBOOK FOR WRITING AND TESTING ENNOISING FROM FILE ###

In [1]:
import librosa as lr
import librosa.display as lrd
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline

conv_or_add = 'conv'
SNR_dB = 10

In [ ]:
### Facts
# The noise is to be applied to the whole file of interest, 
# which will be cut into pieces afterwards and fed in a random order into the network
# The noises last 5' at longest

### Noising - Cases
# (NO) The noise is longer than the file of interest: we can select a trimmed version of the noise by applying a random
# offset and selecting !!! Never happens !!! (the files are 5' long at most)
# (YES) The noise is shorter than the file of interest. Convolution is totally feasible, addition is trickier. 

### Problems?
# Sections of noise 

In [ ]:
### File of interest
x, sr = lr.load('/data/riccardo_datasets/npr_news/train/newscast230841.wav', sr=None, duration=10, offset=2)
s = np.abs(lr.stft(x))

In [ ]:
# Display time + frequency representations + audio
plt.figure(figsize=(14, 8))
plt.subplot(211)
lrd.waveplot(x, sr=sr)
plt.subplot(212)
lrd.specshow(lr.amplitude_to_db(s))

ipd.Audio(x, rate=sr)

In [ ]:
### File taken as Noise
xn, srn = lr.load('/data/riccardo_datasets/demand-noisy_environment-dataset/DWASHING/ch01.wav', sr=None)
sn = np.abs(lr.stft(xn))

# Display time + frequency representations + audio
plt.figure(figsize=(14, 8))
plt.subplot(211)
lrd.specshow(lr.amplitude_to_db(sn))
plt.subplot(212)
lrd.waveplot(xn, sr=srn)
ipd.Audio(xn, rate=srn)


In [ ]:
### Noise params + scaling
snr = 10.0**(SNR_dB/10.0)

# calculate scaling factor for noise
x_rms = x.std()
xn_rms = xn.std()
scaling = x_rms / (snr * xn_rms)
print(x_rms, xn_rms, scaling)
noise = (xn * scaling)

# sum scaled signals
if conv_or_add == 'add':
    noisyx = x + noise
else:
    noisyx = np.convolve(x, noise)

s_noisyx = np.abs(lr.stft(noisyx))

In [ ]:
plt.figure(figsize=(14, 5))
plt.subplot(211)
lrd.specshow(lr.amplitude_to_db(s_noisyx))
plt.subplot(212)
lrd.waveplot(noisyx, sr=sr)
ipd.Audio(noisyx, rate=sr)